### Rozpatrywanym problemem jest opisany na wykładzie problem optymalizacji pracy fabryk okien
#### fabryki mogą jedynie produkować określoną ilość okien w określonym czasie, a dodatkowo wymagają współpracy ze sobą
#### postawionym pytaniem jest jak osiagnąć maksymalne zyski z produkcji okien czyli na produkcje których okien przeznaczyć czas pracy fabryk
zmiennymi problemu sa ilość wyprodukowanych okien danego typu (drewniane lub aluminiowe)  
ograniczeniami sa dostępne, współdzielone czasy fabryk  
algorytm pulp uwzgledniając wszystkie zmienne i ograniczenia optymalizuje problem

![Zdjęcie rozkładu godzin produkcji okien w fabrykach](./images/okna.png)

tabela przedstawia czas potrzebny na produkcję typu okna i dostępne czasy w tygodniu

### znaczenie wiekszosci wywolywanych funkcji i tworzonych zmiennych opisana jest komentarzami w programie

In [5]:
#test
import pulp
print(pulp.__version__)

2.3.1


In [6]:
#import do namespace
from pulp import *

In [22]:
#stworzenie problemu
prob = LpProblem("Problem Okien", LpMaximize)
#zmienne w postaci ilosci wyprodukowanych partii
x1 = LpVariable("Drewniane",0,None,LpInteger)
x2 = LpVariable("Aluminiowe",0,None,LpInteger)
#problem z zyskami
prob +=5000*x1 + 3000*x2, "Calkowity zysk"
#ograniczenia
prob += 0*x1 + 1*x2 <= 4 #fabryka1
prob += 2*x1 + 0*x2 <= 12 #fabryka2
prob += 2*x1 + 3*x2 <= 18 #fabryka3
#x1 += 4
#x2 += 6

#prob += 4*x1 + 4*x2 <= 34


prob.writeLP("ProblemOkien.lp") #zapis do pliku
#rozwiazanie
prob.solve()
#czy rozwiazalo
print("Status:", LpStatus[prob.status])
#dobrane wartosci zmiennych
for v in prob.variables():
    print(v.name, "=", v.varValue)
#wartosc zoptymalizowanej funkcji (zysk)
print("Calkowite zarobki =", value(prob.objective))

Status: Optimal
Aluminiowe = 2.0
Drewniane = 6.0
Calkowite zarobki = 36000.0


### Jako kolejny etap rozwiazywania problemu, została zrealizowana interakcja z problemem, tzn dodanie suwaków pozwalających zmieniać parametry problemu takie jak zysk z produkcji okien (uwzględniany w funkcji optymalizacyjnej) czy ilości godzin pracy fabryk uwzględniane w ograniczeniach

In [33]:
#dorzucic interaktywnosc i tabelke tak jak w pliku formulowanie_zadan, tylko z wlasnym problemem okien
#?? interact
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

style = {'description_width': 'initial'}

zarobek_drewniane_slider=IntSlider(min=0, max = 10000, value = 5000, description="zarobek_drewniane_slider",style = style)
zarobek_aluminiowe_slider=IntSlider(min=0, max = 10000, value = 3000, description="zarobek_aluminiowe_slider",style = style)
godzinyf3aluminiowe_slider=IntSlider(min=0, max = 6, value = 3, description="godzinyf3aluminiowe_slider",style = style)
godzinyf3drewniane_slider=IntSlider(min=0, max = 6, value = 2, description="godzinyf3drewniane_slider",style = style)
godzinyf2aluminiowe_slider=IntSlider(min=0, max = 4, value = 0, description="godzinyf2aluminiowe_slider",style = style)
godzinyf2drewniane_slider=IntSlider(min=0, max = 4, value = 2, description="godzinyf2drewniane_slider",style = style)
godzinyf1aluminiowe_slider=IntSlider(min=0, max = 4, value = 1, description="godzinyf1aluminiowe_slider",style = style)
godzinyf1drewniane_slider=IntSlider(min=0, max = 4, value = 0, description="godzinyf1drewniane_slider",style = style)
godzinyf1max_slider=IntSlider(min=0, max = 4, value = 4, description="godzinyf1max_slider",style = style)
godzinyf2max_slider=IntSlider(min=0, max = 12, value = 12, description="godzinyf2max_slider",style = style)
godzinyf3max_slider=IntSlider(min=0, max = 18, value = 18, description="godzinyf3max_slider",style = style)

def produkcja_okien(zarobek_alum = 3000,
                   zarobek_drew = 5000,
                   godzf3alum=3,
                   godzf3drew=2,
                   godzf2alum=0,
                   godzf2drew=2,
                    godzf1alum=1,
                   godzf1drew=0,
                   godzf3max=18,
                   godzf2max=12,
                   godzf1max=4
                   ):
    #stworzenie problemu
    prob = LpProblem("Problem Okien", LpMaximize)
    #zmienne w postaci ilosci wyprodukowanych partii
    x1 = LpVariable("Drewniane",0,None,LpInteger)
    x2 = LpVariable("Aluminiowe",0,None,LpInteger)
    #problem z zyskami
    prob +=zarobek_drew*x1 + zarobek_alum*x2, "Calkowity zysk"
    #ograniczenia
    prob += godzf1drew*x1 + godzf1alum*x2 <= godzf1max,"czas fabryka 1" #fabryka1
    prob += godzf2drew*x1 + godzf2alum*x2 <= godzf2max,"czas fabryka 2" #fabryka2
    prob += godzf3drew*x1 + godzf3alum*x2 <= godzf3max, "czas fabryka 3" #fabryka3
    #prob += 2*x1 + 0*x2 <= 12 #fabryka2
    #prob += 2*x1 + 3*x2 <= 18 #fabryka3
    
    #rozwiazanie
    prob.solve()
    #czy rozwiazalo
    print("Status:", LpStatus[prob.status])
    #dobrane wartosci zmiennych
    for v in prob.variables():
        print(v.name, "=", v.varValue)
    #wartosc zoptymalizowanej funkcji (zysk)
    print("Calkowite zarobki =", value(prob.objective))
    
    shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
    print("*"*10+"Shadow Prices"+"*"*10)
    print(pd.DataFrame(shadows))

interact(produkcja_okien,
    zarobek_alum=zarobek_aluminiowe_slider,
    zarobek_drew=zarobek_drewniane_slider,
    godzf3alum=godzinyf3aluminiowe_slider,
    godzf3drew=godzinyf3drewniane_slider,
    godzf2alum=godzinyf2aluminiowe_slider,
    godzf2drew=godzinyf2drewniane_slider,
    godzf1alum=godzinyf1aluminiowe_slider,
    godzf1drew=godzinyf1drewniane_slider,
    godzf3max=godzinyf3max_slider,
    godzf2max=godzinyf2max_slider,
    godzf1max=godzinyf1max_slider)

interactive(children=(IntSlider(value=3000, description='zarobek_aluminiowe_slider', max=10000, style=SliderSt…

<function __main__.produkcja_okien(zarobek_alum=3000, zarobek_drew=5000, godzf3alum=3, godzf3drew=2, godzf2alum=0, godzf2drew=2, godzf1alum=1, godzf1drew=0, godzf3max=18, godzf2max=12, godzf1max=4)>

### WNIOSKI
przy powyzszych zalozeniach nawet zwiekszenie
ceny aluminiowych do 10000 to za malo by zmienic ilosc dobieranych okien
dopiero zmniejszenie ilosci godzin potrzebnych na prace nad aluminium przy zwiekszeniu zysku ma wplyw
szczegolnie zmiany w fabryce 2 i 3

shadow price sie nie zmienia, slack (potencjal) natomiast tak
prawdopodobnie jest to kwestia bottlenecka fabrycki 1

